# COVID-19 SNF Dashboard Update

Download data from https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/SNFsCOVID_19.aspx

In [5]:
#Open CDPH's COVID-19 SNF site in browser
import sys
import webbrowser
webbrowser.open('https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/SNFsCOVID_19.aspx', new=2)

True

### ENTER FILE DATES BEFORE RUNNING CELLS

In [6]:
##############################
###SET DATES BEFORE RUNNING###
##############################

#file_date = 'MMDDYY'
file_date = '090620'

#data_date = 'MM/DD/YYYY'
data_date ='09/06/2020'


### DATA PREPARATION

In [7]:
#import modules
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
from arcgis.mapping import WebMap
from arcgis import geometry #use geometry module to project Long,Lat to X and Y
from copy import deepcopy
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy
import json

In [8]:
#connect to ArcGIS account
gis = GIS("pro")

In [9]:
#specify file paths
#paths in
path_in = 'raw_snf_covid_{}.csv'.format(file_date)
path_snf = 'data/snf_basic_info.csv'

#paths out
path_out = 'snf_covid_{}.csv'.format(file_date)

In [10]:
###################################################
###PREP WEBSITE DATA FOR MERGE WITH SPATIAL DATA###
###################################################

#read csv
df = pd.read_csv(path_in)

# replacing blank spaces with '_' in column names 
df.columns =[column.replace(" ", "_") for column in df.columns]

# filtering with query method to eliminate "copy" rows
df.query("Measure_Names == 'Record'", inplace = True) 
  
#delete unnecessary columns from df
del df['COUNTY']
del df['County_(copy)']
del df['FACILITY_NAME']
del df['Measure_Names']
del df['TODAY_POSITIVE_HCW_']
del df['TODAY_POSITIVE_RESIDENTS']
del df['Measure_Values']

#rename columns to match feature layer
df = df.rename(columns = {'Facility_Id':'FACID','COVID-RELATED_HCW_DEATHS_':'HCW_DEATHS_LABEL', 'COVID-RELATED_RESIDENT_DEATHS':'RES_DEATHS_LABEL', 'CUMULATIVE_POSITIVE_HCW_':'HCW_CUM_LABEL', 'CUMULATIVE_POSITIVE_RESIDENTS_':'RES_CUM_LABEL'})

#duplicate columns 
#3 columns for each attribute: label (includes <11), points (<11 as 1 for symbology), stats (<11 as 0 for calcs)
df['HCW_DEATHS_PTS'] = df['HCW_DEATHS_LABEL']
df['HCW_DEATHS_STATS'] = df['HCW_DEATHS_LABEL']

df['RES_DEATHS_PTS'] = df['RES_DEATHS_LABEL']
df['RES_DEATHS_STATS'] = df['RES_DEATHS_LABEL']

df['HCW_CUM_PTS'] = df['HCW_CUM_LABEL']
df['HCW_CUM_STATS'] = df['HCW_CUM_LABEL']

df['RES_CUM_PTS'] = df['RES_CUM_LABEL']
df['RES_CUM_STATS'] = df['RES_CUM_LABEL']

df['DATA_DATE'] = data_date

#format duplicated rows according to rules
#3 columns for each attribute: label (includes <11), points (<11 as 1 for symbology), stats (<11 as 0 for calcs)
for i, row in df.iterrows():
    if(df['HCW_DEATHS_PTS'][i] == '<11'):
        df['HCW_DEATHS_PTS'][i] = 1
    if(df['HCW_DEATHS_STATS'][i] == '<11'):
        df['HCW_DEATHS_STATS'][i] = 0   
    if(df['RES_DEATHS_PTS'][i] == '<11'):
        df['RES_DEATHS_PTS'][i] = 1
    if(df['RES_DEATHS_STATS'][i] == '<11'):
        df['RES_DEATHS_STATS'][i] = 0 
    if(df['HCW_CUM_PTS'][i] == '<11'):
        df['HCW_CUM_PTS'][i] = 1
    if(df['HCW_CUM_STATS'][i] == '<11'):
        df['HCW_CUM_STATS'][i] = 0   
    if(df['RES_CUM_PTS'][i] == '<11'):
        df['RES_CUM_PTS'][i] = 1
    if(df['RES_CUM_STATS'][i] == '<11'):
        df['RES_CUM_STATS'][i] = 0 

#cast columns from string to int for feature layer
df['HCW_DEATHS_PTS'] = df['HCW_DEATHS_PTS'].astype(int)
df['HCW_DEATHS_STATS'] = df['HCW_DEATHS_STATS'].astype(int)
df['RES_DEATHS_PTS'] = df['RES_DEATHS_PTS'].astype(int)
df['RES_DEATHS_STATS'] = df['RES_DEATHS_STATS'].astype(int)
df['HCW_CUM_PTS'] = df['HCW_CUM_PTS'].astype(int)
df['HCW_CUM_STATS'] = df['HCW_CUM_STATS'].astype(int)
df['RES_CUM_PTS'] = df['RES_CUM_PTS'].astype(int)
df['RES_CUM_STATS'] = df['RES_CUM_STATS'].astype(int)

print(len(df))
df.head()

86


,FACID,HCW_DEATHS_LABEL,RES_DEATHS_LABEL,HCW_CUM_LABEL,RES_CUM_LABEL,HCW_DEATHS_PTS,HCW_DEATHS_STATS,RES_DEATHS_PTS,RES_DEATHS_STATS,HCW_CUM_PTS,HCW_CUM_STATS,RES_CUM_PTS,RES_CUM_STATS,DATA_DATE
1,80000056,0,0,<11,<11,0,0,0,0,1,0,1,0,09/06/2020
3,90000102,0,0,0,0,0,0,0,0,0,0,0,0,09/06/2020
5,80000070,0,0,16,<11,0,0,0,0,16,16,1,0,09/06/2020
7,90000029,0,0,0,0,0,0,0,0,0,0,0,0,09/06/2020
9,80000761,<11,0,<11,<11,1,0,0,0,1,0,1,0,09/06/2020


In [11]:
###########################################
###MERGE WEBSITE & SPATIAL (COORDS) DATA###
###########################################

#read SNF file (addresses and coordinates)
snf = pd.read_csv(path_snf)

#merge website and spatial data
snf = snf.merge(df, on = 'FACID')

#########################################
###DETERMINE COUNTY TOTALS AND ADD ROW###
#########################################

#get county wide totals
hcw_deaths_sum = 0
res_deaths_sum = 0
hcw_cum_sum = 0
res_cum_sum = 0

for i,row in snf.iterrows():
    hcw_deaths_sum = hcw_deaths_sum + snf['HCW_DEATHS_STATS'][i]
    res_deaths_sum = res_deaths_sum + snf['RES_DEATHS_STATS'][i]
    hcw_cum_sum = hcw_cum_sum + snf['HCW_CUM_STATS'][i]
    res_cum_sum = res_cum_sum + snf['RES_CUM_STATS'][i]

print(hcw_deaths_sum, res_deaths_sum, hcw_cum_sum, res_cum_sum)

0 81 570 924


In [12]:
#get max deaths and max cumulative cases for point symbology corrections
max_res_cum = snf['RES_CUM_STATS'].max()
print('Max Res Cum: {}'.format(max_res_cum))

max_hcw_cum = snf['HCW_CUM_STATS'].max()
print('Max HCW Cum: {}'.format(max_hcw_cum))

max_cum = 0
if(max_res_cum > max_hcw_cum):
    max_cum = max_res_cum
else:
    max_cum = max_hcw_cum
print('Max Cum: {}'.format(max_cum))

max_res_deaths = snf['RES_DEATHS_STATS'].max()
print('Max Res Deaths: {}'.format(max_res_deaths))

max_hcw_deaths = snf['HCW_DEATHS_STATS'].max()
print('Max HCW Deaths: {}'.format(max_hcw_deaths))

max_deaths = 0
if(max_res_deaths > max_hcw_deaths):
    max_deaths = max_res_deaths
else:
    max_deaths = max_hcw_deaths
print('Max Deaths: {}'.format(max_deaths))

Max Res Cum: 118
Max HCW Cum: 49
Max Cum: 118
Max Res Deaths: 21
Max HCW Deaths: 0
Max Deaths: 21


In [13]:
#create new row for county wide totals
new_row = {'Longitude':-118 , 'Latitude':33 , 'COUNTY': 'SAN DIEGO', 'FACID':999, 'FACILITY_NAME':' ALL SKILLED NURSING FACILITIES', 'SRA':'ALL SRAs', 'ADDRESS':'Totals do not include suppressed data for facilities with <11 confirmed cases or related deaths.', 'DATA_DATE':data_date, 'HCW_DEATHS_LABEL':hcw_deaths_sum, 'RES_DEATHS_LABEL':res_deaths_sum, 'HCW_CUM_LABEL':hcw_cum_sum, 'RES_CUM_LABEL':res_cum_sum, 'HCW_DEATHS_STATS':hcw_deaths_sum, 'RES_DEATHS_STATS':res_deaths_sum, 'HCW_CUM_STATS':hcw_cum_sum, 'RES_CUM_STATS':res_cum_sum, 'HCW_DEATHS_PTS':0, 'RES_DEATHS_PTS':0, 'HCW_CUM_PTS':0, 'RES_CUM_PTS':0}

#append row to dataframe
snf = snf.append(new_row, ignore_index=True)

#save as csv file
snf.to_csv(path_out, index = False)

print(len(snf))
snf.tail(1)

87


,Longitude,Latitude,COUNTY,FACID,FACILITY_NAME,SRA,ADDRESS,HCW_DEATHS_LABEL,RES_DEATHS_LABEL,HCW_CUM_LABEL,RES_CUM_LABEL,HCW_DEATHS_PTS,HCW_DEATHS_STATS,RES_DEATHS_PTS,RES_DEATHS_STATS,HCW_CUM_PTS,HCW_CUM_STATS,RES_CUM_PTS,RES_CUM_STATS,DATA_DATE
86,-118.0,33.0,SAN DIEGO,999,ALL SKILLED NURSING FACILITIES,ALL SRAs,Totals do not include suppressed data for faci...,0,81,570,924,0,0,0,81,0,570,0,924,09/06/2020


### OVERWRITE FEATURE LAYER

In [14]:
#############################
###OVERWRITE FEATURE LAYER###
#############################

#get feature layer containing updated data for maps associated with the COVID-19 dashboards
layer = gis.content.get("55b29be4e904457f994713697f8f073b")
layer

layer_collection = FeatureLayerCollection.fromitem(layer)

#call the overwrite() method which can be accessed using the manager property
layer_collection.manager.overwrite(path_out)


{'success': True}

### FUNCTIONS FOR MAP MODIFICATIONS

Code Reference: https://community.esri.com/groups/arcgis-python-api/blog/2019/04/09/updating-layer-symbology-with-the-arcgis-api-for-python

In [15]:
def get_map (map_id):
    '''
    GET MAP DATA FOR SYMBOLOGY CHANGES
    '''   
    m = gis.content.get(map_id)
    data = m.get_data()    
    print(m)    
    return data

In [16]:
def update_map (map_id, data):
    '''
    UPDATE MAP TO SAVE CHANGES
    '''

    m = gis.content.get(map_id)
    
    # Set the item_properties to include the desired update
    properties = {"text": json.dumps(data)}

    # 'Commit' the updates to the Item
    update = m.update(item_properties=properties)

    # Print item_data to see that changes are reflected
    #new_data = m.get_data()
    #print("***********************NEW DEFINITION**********************")
    #print(json.dumps(new_data, indent=4, sort_keys=True))
    
    return update

### RESIDENTS - CUMULATIVE CASES (res1)

In [17]:
#residents - cumulative cases
res1 = "16dce30762734f579f7849bdafc8c019"

res1_data = get_map(res1)

<Item title:"COVID-19 and San Diego County SNF Residents" type:Web Map owner:jembury8568_SDSUGeo>


In [18]:
#set max value for graduated points symbols
res1_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_cum.item()
res1_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_cum.item()

In [19]:
#update map to save changes
res1_update = update_map(res1, res1_data)
res1_update

True

### RESIDENTS - DEATHS (res2)

In [20]:
#residents - deaths
res2 = "a8319de392f64e4c836d42190ed10205"

res2_data = get_map(res2)

<Item title:"COVID-19 and San Diego County SNF Resident Deaths" type:Web Map owner:jembury8568_SDSUGeo>


In [21]:
#set max value for graduated points symbols
res2_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_deaths.item()
res2_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_deaths.item()

In [22]:
#update map to save changes
res2_update = update_map(res2, res2_data)
res2_update

True

### HCW - CUMULATIVE CASES (hcw1)

In [23]:
#healthcare workers - cumulative cases
hcw1 = "11543f486f414409b3106b98eb3e5fe4"

hcw1_data = get_map(hcw1)

<Item title:"COVID-19 and San Diego County SNF Healthcare Workers" type:Web Map owner:jembury8568_SDSUGeo>


In [24]:
#set max value for graduated points symbols
hcw1_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_cum.item()
hcw1_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_cum.item()

In [25]:
#update map to save changes
hcw1_update = update_map(hcw1, hcw1_data)
hcw1_update

True

### HCW - DEATHS (hcw2)

In [26]:
#healthcare workers - deaths
hcw2 = "d2cbaba1d61e43dab61aeb2fb2bd26de"

hcw2_data = get_map(hcw2)

<Item title:"COVID-19 and San Diego County SNF Healthcare Worker Deaths" type:Web Map owner:jembury8568_SDSUGeo>


In [27]:
#set max value for graduated points symbols
hcw2_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_deaths.item()
hcw2_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_deaths.item()

In [28]:
#update map to save changes
hcw2_update = update_map(hcw2, hcw2_data)
hcw2_update

True

### VERIFY COVID-19 SNF DASHBOARD

In [29]:
#Open COVID-19 SNF dashboard in browser
webbrowser.open('https://arcg.is/0bjCe1', new=2)

True